# Sequence data

- 시퀀스 데이터는 순서가 있고 이것이 중요한 데이터로
- 자연어, 유전, 시계열 자료 등이 이런 자료형에 속한다.
  
    
- 모델링시 중요한 점은 이 순서의 의미를 반영해주어야 한다는 점이고
- 가장 각광받는 RNN과 CNN에 대해서 소개한다.

## RNN 

- 재귀적/순한 신경망을 뜻하며 데이터를 다룰 시에 히든 노드가 다음 히든 노드에 영향을 주는 모형
- 마찬가지로 역전파 알고리즘을 이용하여 업데이트하나 생각보다 코드 구현이 쉽지 않음(BPTT)
- 심플한 RNN보다는 LSTM이 엄청난 각광을 받았고 한국교수가 제안한 GRU도 유명함
- 코드 구성시 얼만큼의 시간을 고려할지를 인풋으로 넣어줘야함
    - function만들어서 가능
    - 자료형을 변형시켜서 가능
    

## CNN

- 본래 이미지를 대상으로하는 분석에서 엄청난 획을 그은 분석 기법
- 이미지관련 분석은 CNN전과 후로 나눌 수 있을 듯
- 시퀀스데이터에서도 비슷한 개념을 활요하여 접근 가능

In [1]:
from sklearn.datasets import load_boston
import numpy as np
import keras
from keras import models, layers
from keras.callbacks import EarlyStopping
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')

d = load_boston()

Using TensorFlow backend.


In [2]:
d.data[0]

array([6.320e-03, 1.800e+01, 2.310e+00, 0.000e+00, 5.380e-01, 6.575e+00,
       6.520e+01, 4.090e+00, 1.000e+00, 2.960e+02, 1.530e+01, 3.969e+02,
       4.980e+00])

In [3]:
temp_data = (d.data-np.mean(d.data))/np.std(d.data)

In [4]:
data = []
lags = 10

for i in range(lags, len(temp_data)):
    temp = temp_data[max(0, i-lags):i]
    data.append(temp)
    
data = np.array(data)

In [136]:
keras.backend.clear_session()

input_layer = layers.Input(shape=(lags, d.data.shape[1]))

l11 = layers.SimpleRNN(32, return_sequences=False )(input_layer)
l12 = layers.LSTM(32, return_sequences=False)(input_layer)

l2 = layers.concatenate([l11, l12])

l3 = layers.Dense(32, activation='relu')(l2)

out_layer = layers.Dense(1, activation='linear')(l3)

model = models.Model(input_layer, out_layer)

model.compile(optimizer='adam',
             loss = 'mse',
             metrics=['mae']
             )

In [137]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10, 13)       0                                            
__________________________________________________________________________________________________
simple_rnn_1 (SimpleRNN)        (None, 32)           1472        input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 32)           5888        input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 64)           0           simple_rnn_1[0][0]               
                                                                 lstm_1[0][0]               

In [138]:
# from keras.utils import plot_model
# plot_model(model, to_file='model.png')

In [139]:
es = EarlyStopping(patience=20, monitor='val_loss', restore_best_weights=True)

In [140]:
model.fit(data, d.target[10:],
         epochs=400,
         validation_split=0.3,
         callbacks=[es])

Train on 347 samples, validate on 149 samples
Epoch 1/400
347/347 [==============================] - 1s 1ms/step - loss: 621.6038 - mae: 23.4448 - val_loss: 271.0958 - val_mae: 14.3012
Epoch 2/400
347/347 [==============================] - 0s 217us/step - loss: 523.4302 - mae: 21.2768 - val_loss: 217.0744 - val_mae: 12.3091
Epoch 3/400
347/347 [==============================] - 0s 210us/step - loss: 431.8189 - mae: 18.9950 - val_loss: 160.2973 - val_mae: 9.8632
Epoch 4/400
347/347 [==============================] - 0s 224us/step - loss: 335.4144 - mae: 16.2233 - val_loss: 108.2965 - val_mae: 7.2378
Epoch 5/400
347/347 [==============================] - 0s 213us/step - loss: 240.4238 - mae: 12.9947 - val_loss: 75.0190 - val_mae: 5.6117
Epoch 6/400
347/347 [==============================] - 0s 198us/step - loss: 162.9591 - mae: 9.7059 - val_loss: 66.7593 - val_mae: 5.7422
Epoch 7/400
347/347 [==============================] - 0s 187us/step - loss: 109.5975 - mae: 7.1459 - val_loss: 82.02

In [ ]:
model.evaluate(data, d.target[lags:])

In [11]:
keras.backend.clear_session()

input_layer = layers.Input(shape=(lags, d.data.shape[1]))

x1 = layers.Conv1D(filters=32, kernel_size=3, padding='same')(input_layer)
x1 = layers.MaxPooling1D()(x1)

x2 = layers.Conv1D(filters=16, kernel_size=5, padding='same')(input_layer)
x2 = layers.MaxPooling1D()(x2)

x = layers.concatenate([x1, x2])
x = layers.GlobalAveragePooling1D()(x)

x = layers.Dense(32, activation='relu')(x)

out_layer = layers.Dense(1, activation='linear')(x)

model = models.Model(input_layer, out_layer)

model.compile(optimizer='adam',
             loss = 'mse',
             metrics=['mae']
             )

In [12]:
data2 = np.reshape(data, (496, 10, 13))

In [13]:
model.fit(data2, d.target[lags:],
         epochs=400,
         validation_split=0.3,
         callbacks=[es]
         )

Train on 347 samples, validate on 149 samples
Epoch 1/400
347/347 [==============================] - 0s 426us/step - loss: 635.9276 - mae: 23.7645 - val_loss: 273.7583 - val_mae: 14.4208
Epoch 2/400
347/347 [==============================] - 0s 63us/step - loss: 519.0238 - mae: 21.0810 - val_loss: 177.9767 - val_mae: 10.6652
Epoch 3/400
347/347 [==============================] - 0s 63us/step - loss: 391.8508 - mae: 17.8494 - val_loss: 101.6320 - val_mae: 6.9895
Epoch 4/400
347/347 [==============================] - 0s 59us/step - loss: 256.3368 - mae: 13.4617 - val_loss: 65.3730 - val_mae: 5.5638
Epoch 5/400
347/347 [==============================] - 0s 61us/step - loss: 136.3793 - mae: 8.3722 - val_loss: 120.5859 - val_mae: 9.2711
Epoch 6/400
347/347 [==============================] - 0s 64us/step - loss: 79.3806 - mae: 6.3803 - val_loss: 234.7483 - val_mae: 14.2264
Epoch 7/400
347/347 [==============================] - 0s 56us/step - loss: 78.4466 - mae: 7.2048 - val_loss: 256.6180 -

In [14]:
model.evaluate(data, d.target[lags:])

496/496 [==============================] - 0s 20us/step


[147.31325623296922, 9.047750473022461]

In [15]:
keras.backend.clear_session()

input_layer = layers.Input(shape=(lags, d.data.shape[1]))

x = layers.Conv1D(filters=32, kernel_size=3)(input_layer)
x = layers.AveragePooling1D()(x)
x = layers.Conv1D(filters=16, kernel_size=3)(x)
x = layers.AveragePooling1D()(x)
# x = layers.GlobalAveragePooling1D()(x)

x = layers.Bidirectional(layers.GRU(16, return_sequences=False))(x)

x = layers.Dense(32, activation='relu')(x)

out_layer = layers.Dense(1, activation='linear')(x)

model = models.Model(input_layer, out_layer)

model.compile(optimizer='adam',
             loss = 'mse',
             metrics=['mae']
             )

In [16]:
model.fit(data2, d.target[lags:],
         epochs=400,
         validation_split=0.3,
         callbacks=[es]
         )

Train on 347 samples, validate on 149 samples
Epoch 1/400
347/347 [==============================] - 1s 2ms/step - loss: 666.2963 - mae: 24.3800 - val_loss: 300.8329 - val_mae: 15.2851
Epoch 2/400
347/347 [==============================] - 0s 111us/step - loss: 607.6659 - mae: 23.1532 - val_loss: 260.6671 - val_mae: 13.9020
Epoch 3/400
347/347 [==============================] - 0s 103us/step - loss: 544.8543 - mae: 21.7516 - val_loss: 222.5266 - val_mae: 12.4518
Epoch 4/400
347/347 [==============================] - 0s 106us/step - loss: 477.2533 - mae: 20.1269 - val_loss: 179.8298 - val_mae: 10.6327
Epoch 5/400
347/347 [==============================] - 0s 106us/step - loss: 398.9569 - mae: 18.0986 - val_loss: 138.1699 - val_mae: 8.6544
Epoch 6/400
347/347 [==============================] - 0s 105us/step - loss: 324.9789 - mae: 15.9031 - val_loss: 107.1475 - val_mae: 7.1256
Epoch 7/400
347/347 [==============================] - 0s 103us/step - loss: 259.5157 - mae: 13.6671 - val_loss:

In [17]:
model.evaluate(data, d.target[lags:])

496/496 [==============================] - 0s 36us/step


[111.40012885678199, 7.49119758605957]